In [19]:
from src.solver import Solver0D

In [20]:
x = Solver0D()
x.read_solver_file('../data/healthy/0080_0001/base_solver_files/0080_0001_model.in')

In [21]:
tree = x.get_vessel_tree()

In [22]:
tree.children[0].side

'rpa'

In [24]:
def compute_total_resistance(cur_node: Solver0D.Node):
    ''' computes total resistance/stenosis coefficients of a tree recursively
    '''
    
    if not cur_node.children:
        r_p = cur_node.vessel_info[0]['zero_d_element_values']['R_poiseuille']
        sc = cur_node.vessel_info[0]['zero_d_element_values']['stenosis_coefficient']
        return r_p, sc
    
    total_inv_res = 0
    total_inv_sc = 0
    for child_node in cur_node.children:
        r_p_child, sc_child = compute_total_resistance(child_node)
        if r_p_child != 0:
            total_inv_res += 1/r_p_child
        if sc_child != 0:
            total_inv_sc += 1/sc_child
    total_child_res = 1/total_inv_res
    total_child_sc = 1/ total_inv_sc
    
    r_p = cur_node.vessel_info[0]['zero_d_element_values']['R_poiseuille'] + total_child_res
    sc = cur_node.vessel_info[0]['zero_d_element_values']['stenosis_coefficient'] + total_child_sc
    return r_p, sc


    
        

def compute_lpa_rpa_resistances(dummy_solver: Solver0D ):
    ''' computes LPA and RPA resistances from each branch
    '''
        
    # get tree
    vessel_tree = dummy_solver.get_vessel_tree()
    assert len(vessel_tree.children) == 2, 'Pulmonary vasculature should only branch into 2 sections, LPA and RPA'
    
    first_branch = vessel_tree.children[0]
    second_branch = vessel_tree.children[1]
    
    # follow the first branch till it finishes to determine which is LPA and which is RPA
    if first_branch.side == 'lpa':
        lpa = first_branch
        rpa = second_branch
    else:
        print(True)
        lpa = second_branch
        rpa = first_branch
    
    lpa_res, lpa_sc = compute_total_resistance(lpa)
    rpa_res, rpa_sc = compute_total_resistance(rpa)
    
    
    return lpa_res, lpa_sc, rpa_res, rpa_sc    

In [25]:
compute_lpa_rpa_resistances(x)

True


(4.301938160248982,
 0.006720849186968765,
 5.578715289718572,
 0.004715818848199389)

In [27]:
rpa = tree.children[0]
max_gen = 0
max_node = None
for node in x.tree_bfs_iterator(rpa):
    if node.generation > max_gen:
        max_gen = node.generation
        max_node = node

In [29]:
max_node.__dict__

{'vess_id': [139],
 'vessel_info': [{'vessel_id': 139,
   'vessel_length': 0.1133163787610899,
   'vessel_name': 'branch97_seg0',
   'zero_d_element_type': 'BloodVessel',
   'zero_d_element_values': {'C': 1.184502234869842e-08,
    'L': 1.4363668337394515,
    'R_poiseuille': 16.290190768477277,
    'stenosis_coefficient': 0.009707247347680057}}],
 'parent': 138,
 'children': [<src.solver.Solver0D.VesselNode at 0x7fc612dde550>],
 'generation': 10,
 'side': 'rpa'}

In [30]:
import os

In [35]:
for x in os.walk(top = '../data/healthy/0080_0001/base_solver_files/'):
    print(x)

('../data/healthy/0080_0001/base_solver_files/', ['artificial_stenosis', 'tuning_dir'], ['0080_0001_model.in', '0080_0001_model_branch_results.csv', 'inlet_face_names.dat', '0080_0001_model_branch_results.npy', 'CapInfo', 'outlet_mapping.dat', 'inflow.png', 'inflow.flow', 'model_centerlines.vtp', '0080_0001_model_inlet_pressures.png', 'inlet_mapping.dat', 'rcrt.dat', 'outlet_face_names.dat'])
('../data/healthy/0080_0001/base_solver_files/artificial_stenosis', ['proximal', '0'], [])
('../data/healthy/0080_0001/base_solver_files/artificial_stenosis/proximal', [], ['stenosis_vessels.dat', '0080_0001_model_proximal_sten.in', '0080_0001_model_proximal_sten_branch_results.csv', '0080_0001_model_proximal_sten_branch_results.npy', '0080_0001_model_proximal_sten_inlet_pressures.png'])
('../data/healthy/0080_0001/base_solver_files/artificial_stenosis/0', [], ['0080_0001_model_art_sten_inlet_pressures.png', 'stenosis_vessels.dat', '0080_0001_model_art_sten.in', '0080_0001_model_art_sten_branch_re

In [45]:
import numpy as np

In [48]:
x = np.loadtxt('../data/healthy/0082_0001/base_solver_files/inflow.flow')

In [49]:
np.trapz(x[:,1], x[:,0])/ (x[-1, 0] - x[0, 0]) * 60/1000

6.897197289978845

In [41]:
x = np.array([1, 2, 3, 4,])


In [44]:
x[np.where(x == 4)][0]

4